In [1]:
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import datasets, linear_model
from datetime import datetime
import gc
%matplotlib inline
from IPython.display import display, HTML
from pprint import pprint
import xgboost as xgb

## Usage!!!
- Always change maxVal if iterative feature is added..5, for np.nan's add 4, then 3.. maxVal calculates according to this..

In [2]:
class ConfigElements:
    def __init__(self, lag, nameAndGroups, targetVariable="", deleteColumns = False):
        self.lag = lag
        self.nameAndGroups = nameAndGroups
        #If there is target variable, then 5 4 3 2 1, fill the np.nans..Else hold them all in Dataframe..
        self.targetVariable = targetVariable
        self.deleteColumns = deleteColumns

In [18]:
configLag0Target1 = ConfigElements(0,[ ("A",["Semana","Agencia_ID"],["count","count"]),("B",["Agencia_ID"],["mean"]),
    ("C",["Semana","Agencia_ID"],["count","mean"])], "lag0tar1", False)
print  configLag0Target1.__dict__

{'nameAndGroups': [('A', ['Semana', 'Agencia_ID'], ['count', 'count']), ('B', ['Agencia_ID'], ['mean']), ('C', ['Semana', 'Agencia_ID'], ['count', 'mean'])], 'lag': 0, 'targetVariable': 'lag0tar1', 'deleteColumns': False}


In [19]:
configLag0Target0 = ConfigElements(0,[ ("D",["Semana","Agencia_ID"],["count","count"]),("E",["Agencia_ID"],["mean"]),
    ("F",["Semana","Agencia_ID"],["count","mean"])])
print  configLag0Target0.__dict__

{'nameAndGroups': [('D', ['Semana', 'Agencia_ID'], ['count', 'count']), ('E', ['Agencia_ID'], ['mean']), ('F', ['Semana', 'Agencia_ID'], ['count', 'mean'])], 'lag': 0, 'targetVariable': '', 'deleteColumns': False}


In [27]:
configLag1Target1 = ConfigElements(1,[ ("SA2",["Semana","Agencia_ID"],["count"]),("L",["Semana","Agencia_ID"],["mean"])
                                     ],"lag1tar1")
print  configLag1Target1.__dict__

{'nameAndGroups': [('SA2', ['Semana', 'Agencia_ID'], ['count']), ('L', ['Semana', 'Agencia_ID'], ['mean'])], 'lag': 1, 'targetVariable': 'lag1tar1', 'deleteColumns': False}


In [39]:
configLag1Target1F = ConfigElements(1,[ ("SAP1CC",["Semana","Agencia_ID","Producto_ID"],
                                    ["count"]),("SAP1MM",["Semana","Agencia_ID","Producto_ID"],["mean"])],"Lag1Target1F",True
                                     )
print  configLag0Target1.__dict__

{'nameAndGroups': [('A', ['Semana', 'Agencia_ID'], ['count', 'count']), ('B', ['Agencia_ID'], ['mean']), ('C', ['Semana', 'Agencia_ID'], ['count', 'mean'])], 'lag': 0, 'targetVariable': 'lag0tar1', 'deleteColumns': False}


##  Test for filling missing values that are already on a column, no need to create a target variable..

In [8]:
#configWithoutLag2 = ConfigElements(0,[ ("A",["Agencia_ID"]), ("Ca",["Canal_ID"]), ("R",["Ruta_SAK"]), 
#    ("Cl",["Cliente_ID"]),("P",["Producto_ID"]),("ClP",["Cliente_ID","Producto_ID"])   ], "lag2par1")
#print  configWithLag.__dict__

In [7]:
class FeatureEngineering:
    #ValidationStart = 0
    #lag = 0
    #trainHdfPath = ""
    #trainHdfFile = ""
    #testHdfPath = ""
    #testHdfFile = ""
    #train = pd.DataFrame()
    #test = pd.DataFrame()
    
    def __init__(self, ValidationStart, ValidationSmallerThan, trainHdfPath, trainHdfFile, testHdfPath, testHdfFile, 
                 testTypes, trainTypes, trainCsvPath, testCsvPath, maxLag=0):
        self.ValidationStart = ValidationStart
        self.ValidationSmallerThan = ValidationSmallerThan
        self.maxLag = maxLag
        self.trainHdfPath = trainHdfPath
        self.trainHdfFile = trainHdfFile
        self.testHdfPath = testHdfPath
        self.testHdfFile = testHdfFile
        self.testTypes = testTypes
        self.trainTypes = trainTypes
        self.trainCsvPath = trainCsvPath
        self.testCsvPath = testCsvPath
        
    @staticmethod
    def __printDataFrameBasics__(data):
        display(data.head(2))
        #print data.dtypes
        gc.collect()
        print(data.info(memory_usage=True))
        
    @staticmethod    
    def __changeIndexTypeToLowerMemory(data):
        ##########
        #This is very critical, i accept max number is 2^32. Also, if don't do that, memory gets so much higher..
        ##########
        data.index = data.index.astype('uint32')
        
    def ReadHdf(self, trainOrTestOrBoth):
        '''Reads and holds Df in object memory'''
        if trainOrTestOrBoth == '':
            raise "qweqweq"
            
        if trainOrTestOrBoth == 'train' or trainOrTestOrBoth=='both':
            self.train = pd.read_hdf(self.trainHdfPath,self.trainHdfFile)
            FeatureEngineering.__changeIndexTypeToLowerMemory(self.train)
            FeatureEngineering.__printDataFrameBasics__(self.train)
        if trainOrTestOrBoth == 'test' or trainOrTestOrBoth=='both':
            self.test = pd.read_hdf(self.testHdfPath,self.testHdfFile)
            FeatureEngineering.__changeIndexTypeToLowerMemory(self.test)
            FeatureEngineering.__printDataFrameBasics__(self.test)
        
    def ReadCsv(self, trainOrTestOrBoth):
        '''Reads and holds Df in object memory'''
        if trainOrTestOrBoth == '':
            raise "qweqweq"  
        if trainOrTestOrBoth == 'train' or trainOrTestOrBoth=='both':
            self.train = pd.read_csv(self.trainCsvPath, usecols=self.trainTypes.keys(), dtype=self.trainTypes)
            #__changeIndexTypeToLowerMemory(self.train)
            FeatureEngineering.__printDataFrameBasics__(self.train)
        if trainOrTestOrBoth == 'test' or trainOrTestOrBoth=='both':
            self.test = pd.read_csv(self.testCsvPath, usecols=self.testTypes.keys(), dtype=self.testTypes)
            #__changeIndexTypeToLowerMemory(self.test)
            FeatureEngineering.__printDataFrameBasics__(self.test)

    @staticmethod
    def ConvertCsvToHdf(csvPath, HdfPath, HdfName, ColumnTypeDict ):
        tempDf = pd.read_csv(csvPath, usecols=ColumnTypeDict.keys(), dtype=ColumnTypeDict)
        tempDf.to_hdf(HdfPath, HdfName, format='t')
        del tempDf
        gc.collect()
        print "ConvertCsvToHdf is done.."

    def Preprocess(self, trainOrTestOrBoth, columnFunctionTypeList):
        '''columnFunctionTypeList = [ ['C1',Func1,Type], ['C2',Func2,Type],..    ]'''
        for column, func, localType in columnFunctionTypeList:
            if trainOrTestOrBoth == 'train' or trainOrTestOrBoth=='both':
                self.train.loc[:,column] =  self.train[column].apply(func).astype(localType)
            elif trainOrTestOrBoth == 'test' or trainOrTestOrBoth=='both':
                self.test.loc[:,column] =  self.test[column].apply(func).astype(localType)
            
    def SaveDataFrameToHdf(self,trainOrTestOrBoth):
        if trainOrTestOrBoth == 'train' or trainOrTestOrBoth=='both':
            self.train.to_hdf(self.trainHdfPath, self.trainHdfFile, format='t')
        if trainOrTestOrBoth == 'test' or trainOrTestOrBoth=='both':
            self.test.to_hdf(self.testHdfPath, self.testHdfFile, format='t')
        
    def AddDemandaGeneralMean(self):
        DemandaMeanWithoutLag = self.train.loc[self.train.loc[:,'Semana'] < self.ValidationStart]
        self.train.loc[:,"DemandaGeneralMean"] = DemandaMeanWithoutLag['Demanda_uni_equil'].mean()
        self.train.loc[:,"DemandaGeneralMean"] = self.train.loc[:,"DemandaGeneralMean"].astype('float32')
        display(self.train)
        del DemandaMeanWithoutLag
        gc.collect()
        
        
       ## ("A",["Semana","Agencia_ID"],["count","count"])
    def AddConfigurableFeaturesToTrain(self, config):
        if config.lag > self.maxLag:
            self.maxLag = config.lag
        
        lag = 0 if config.lag == 0 else config.lag-1
        tempData = self.train.loc[self.train.loc[:,'Semana'] < self.ValidationStart - lag]
        #display(tempData)
        if(config.lag != 0):
            tempData.loc[:,'Semana'] = tempData.loc[:,'Semana'].apply(lambda x:x + config.lag)
        #display(tempData)
        
        #Means iterative.. eliminate as long as np.nan exists..If there is already one, don't create, use the existing
        if config.targetVariable != "" and  config.targetVariable not in self.train.columns:
            self.train.loc[:,config.targetVariable] = np.nan
        
        for name,groups,aggregate in config.nameAndGroups:
            if name not in self.train.columns:
                print "{} is not in columns..".format(name)
                
                ##First, select columns not to fill memory..then groupby..
                
#                groupedDataframe =  pd.DataFrame({name:tempData[groups+['Demanda_uni_equil']].groupby(groups)
 #                   ['Demanda_uni_equil'].mean().astype('float32')})
                
                
                
                groupedDataframe = tempData[groups+['Demanda_uni_equil']].groupby(groups).agg(aggregate[0])
                #groupedDataframe.columns = groupedDataframe.columns.droplevel(0)
                groupedDataframe.columns = [name]
                
                #This is means of the counts of the semana-columns tuples!..!!!
                #If no lag and mean, mean of the columns without semana!!..
                #If there is lag and count, count of the columns x weeks before
                #If there is lag and mean, mean of the columns x weeks before
                #if(config.lag == 0 and aggregate == "count"):
                if(len(aggregate)>1):
                    groupedDataframe.reset_index(inplace=True)
                    groupedDataframe.drop("Semana",axis=1, inplace=True)
                    groups = groups[1:]
                    groupedDataframe = groupedDataframe.groupby(groups).agg(aggregate[1])
                    groupedDataframe.columns = [name]
                    gc.collect()
                
                display(groupedDataframe)
                self.train = self.train.merge( groupedDataframe, left_on=groups,
                                  right_index=True, how='left', sort=False,copy=False)
                del groupedDataframe
                gc.collect()
            else:
                print "{} is in columns..".format(name)
            
            display(self.train)
            
            #Means iterative..!!!!!
            if config.targetVariable != "":
                self.train.loc[pd.isnull(self.train[config.targetVariable]), 
                    config.targetVariable] = self.train.loc[pd.isnull(self.train[config.targetVariable]), name]
                count = self.train.loc[self.train['Semana'] >= self.ValidationStart,config.targetVariable].isnull().sum()
                print "Count of missing numbers after {} in validation part in column {} is {}".format(name, 
                    config.targetVariable,str(count))
                #display(self.train)
                #If column is already in Dataframe and we want to fill target variable, this deletes columns!!!
                if(config.deleteColumns):
                    self.train.drop(name, axis=1, inplace=True)
                gc.collect()
                if count == 0:
                    break
        del tempData
        display(self.train)        
        gc.collect()
        return 
    def DeleteLaggedWeeksFromTrain(self):
        self.train = self.train.loc[self.train.loc[:,'Semana']>= 3 + self.maxLag]
        gc.collect()
        display(self.train)

## Test1

In [31]:
FE.AddConfigurableFeaturesToTrain(configLag0Target1)

A is in columns..


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_proxima,Demanda_uni_equil,lag0tar1,A,B,C,D,E,F,lag1tar1,SA2,L
0,3,1111,1,4402,50789,32846,2,10.920000,0,1.098612,1,1,1.098612,1.000000,1,1.098612,1.000000,NaN,NaN,NaN
1,3,1112,1,2103,1209569,36748,4,46.119999,0,1.609438,1,1,1.609438,1.000000,1,1.609438,1.000000,NaN,NaN,NaN
2,4,1116,1,1603,399077,35147,3,61.500000,0,2.079442,2,2,1.732868,1.000000,2,1.732868,1.000000,1,1,1.386294
3,3,1116,1,2124,174266,36748,3,34.590000,0,1.386294,2,2,1.732868,1.000000,2,1.732868,1.000000,NaN,NaN,NaN
4,4,1117,1,1061,641223,1129,5,88.000000,0,1.791759,3,3,1.069167,1.333333,3,1.069167,1.333333,1,1,1.098612
5,4,1117,1,1272,1649797,1109,1,15.010000,0,0.693147,3,3,1.069167,1.333333,3,1.069167,1.333333,1,1,1.098612
6,5,1117,1,1272,2270968,41938,1,9.910000,0,0.693147,3,3,1.069167,1.333333,3,1.069167,1.333333,2,2,1.242453
7,3,1117,1,1281,358759,31423,2,21.379999,0,1.098612,3,3,1.069167,1.333333,3,1.069167,1.333333,NaN,NaN,NaN
8,3,1119,1,1457,660918,1250,2,15.280000,0,1.098612,1,1,1.098612,1.000000,1,1.098612,1.000000,NaN,NaN,NaN
9,3,1138,1,1203,402789,972,1,18.980000,0,0.693147,3,3,1.416165,1.000000,3,1.416165,1.000000,NaN,NaN,NaN


Count of missing numbers after A in validation part in column lag0tar1 is 1
B is in columns..


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_proxima,Demanda_uni_equil,lag0tar1,A,B,C,D,E,F,lag1tar1,SA2,L
0,3,1111,1,4402,50789,32846,2,10.920000,0,1.098612,1,1,1.098612,1.000000,1,1.098612,1.000000,NaN,NaN,NaN
1,3,1112,1,2103,1209569,36748,4,46.119999,0,1.609438,1,1,1.609438,1.000000,1,1.609438,1.000000,NaN,NaN,NaN
2,4,1116,1,1603,399077,35147,3,61.500000,0,2.079442,2,2,1.732868,1.000000,2,1.732868,1.000000,1,1,1.386294
3,3,1116,1,2124,174266,36748,3,34.590000,0,1.386294,2,2,1.732868,1.000000,2,1.732868,1.000000,NaN,NaN,NaN
4,4,1117,1,1061,641223,1129,5,88.000000,0,1.791759,3,3,1.069167,1.333333,3,1.069167,1.333333,1,1,1.098612
5,4,1117,1,1272,1649797,1109,1,15.010000,0,0.693147,3,3,1.069167,1.333333,3,1.069167,1.333333,1,1,1.098612
6,5,1117,1,1272,2270968,41938,1,9.910000,0,0.693147,3,3,1.069167,1.333333,3,1.069167,1.333333,2,2,1.242453
7,3,1117,1,1281,358759,31423,2,21.379999,0,1.098612,3,3,1.069167,1.333333,3,1.069167,1.333333,NaN,NaN,NaN
8,3,1119,1,1457,660918,1250,2,15.280000,0,1.098612,1,1,1.098612,1.000000,1,1.098612,1.000000,NaN,NaN,NaN
9,3,1138,1,1203,402789,972,1,18.980000,0,0.693147,3,3,1.416165,1.000000,3,1.416165,1.000000,NaN,NaN,NaN


Count of missing numbers after B in validation part in column lag0tar1 is 1
C is in columns..


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_proxima,Demanda_uni_equil,lag0tar1,A,B,C,D,E,F,lag1tar1,SA2,L
0,3,1111,1,4402,50789,32846,2,10.920000,0,1.098612,1,1,1.098612,1.000000,1,1.098612,1.000000,NaN,NaN,NaN
1,3,1112,1,2103,1209569,36748,4,46.119999,0,1.609438,1,1,1.609438,1.000000,1,1.609438,1.000000,NaN,NaN,NaN
2,4,1116,1,1603,399077,35147,3,61.500000,0,2.079442,2,2,1.732868,1.000000,2,1.732868,1.000000,1,1,1.386294
3,3,1116,1,2124,174266,36748,3,34.590000,0,1.386294,2,2,1.732868,1.000000,2,1.732868,1.000000,NaN,NaN,NaN
4,4,1117,1,1061,641223,1129,5,88.000000,0,1.791759,3,3,1.069167,1.333333,3,1.069167,1.333333,1,1,1.098612
5,4,1117,1,1272,1649797,1109,1,15.010000,0,0.693147,3,3,1.069167,1.333333,3,1.069167,1.333333,1,1,1.098612
6,5,1117,1,1272,2270968,41938,1,9.910000,0,0.693147,3,3,1.069167,1.333333,3,1.069167,1.333333,2,2,1.242453
7,3,1117,1,1281,358759,31423,2,21.379999,0,1.098612,3,3,1.069167,1.333333,3,1.069167,1.333333,NaN,NaN,NaN
8,3,1119,1,1457,660918,1250,2,15.280000,0,1.098612,1,1,1.098612,1.000000,1,1.098612,1.000000,NaN,NaN,NaN
9,3,1138,1,1203,402789,972,1,18.980000,0,0.693147,3,3,1.416165,1.000000,3,1.416165,1.000000,NaN,NaN,NaN


Count of missing numbers after C in validation part in column lag0tar1 is 1


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_proxima,Demanda_uni_equil,lag0tar1,A,B,C,D,E,F,lag1tar1,SA2,L
0,3,1111,1,4402,50789,32846,2,10.920000,0,1.098612,1,1,1.098612,1.000000,1,1.098612,1.000000,NaN,NaN,NaN
1,3,1112,1,2103,1209569,36748,4,46.119999,0,1.609438,1,1,1.609438,1.000000,1,1.609438,1.000000,NaN,NaN,NaN
2,4,1116,1,1603,399077,35147,3,61.500000,0,2.079442,2,2,1.732868,1.000000,2,1.732868,1.000000,1,1,1.386294
3,3,1116,1,2124,174266,36748,3,34.590000,0,1.386294,2,2,1.732868,1.000000,2,1.732868,1.000000,NaN,NaN,NaN
4,4,1117,1,1061,641223,1129,5,88.000000,0,1.791759,3,3,1.069167,1.333333,3,1.069167,1.333333,1,1,1.098612
5,4,1117,1,1272,1649797,1109,1,15.010000,0,0.693147,3,3,1.069167,1.333333,3,1.069167,1.333333,1,1,1.098612
6,5,1117,1,1272,2270968,41938,1,9.910000,0,0.693147,3,3,1.069167,1.333333,3,1.069167,1.333333,2,2,1.242453
7,3,1117,1,1281,358759,31423,2,21.379999,0,1.098612,3,3,1.069167,1.333333,3,1.069167,1.333333,NaN,NaN,NaN
8,3,1119,1,1457,660918,1250,2,15.280000,0,1.098612,1,1,1.098612,1.000000,1,1.098612,1.000000,NaN,NaN,NaN
9,3,1138,1,1203,402789,972,1,18.980000,0,0.693147,3,3,1.416165,1.000000,3,1.416165,1.000000,NaN,NaN,NaN


## Test 2

In [30]:
FE.AddConfigurableFeaturesToTrain(configLag0Target0 )

D is in columns..


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_proxima,Demanda_uni_equil,lag0tar1,A,B,C,D,E,F,lag1tar1,SA2,L
0,3,1111,1,4402,50789,32846,2,10.920000,0,1.098612,1,1,1.098612,1.000000,1,1.098612,1.000000,NaN,NaN,NaN
1,3,1112,1,2103,1209569,36748,4,46.119999,0,1.609438,1,1,1.609438,1.000000,1,1.609438,1.000000,NaN,NaN,NaN
2,4,1116,1,1603,399077,35147,3,61.500000,0,2.079442,2,2,1.732868,1.000000,2,1.732868,1.000000,1,1,1.386294
3,3,1116,1,2124,174266,36748,3,34.590000,0,1.386294,2,2,1.732868,1.000000,2,1.732868,1.000000,NaN,NaN,NaN
4,4,1117,1,1061,641223,1129,5,88.000000,0,1.791759,3,3,1.069167,1.333333,3,1.069167,1.333333,1,1,1.098612
5,4,1117,1,1272,1649797,1109,1,15.010000,0,0.693147,3,3,1.069167,1.333333,3,1.069167,1.333333,1,1,1.098612
6,5,1117,1,1272,2270968,41938,1,9.910000,0,0.693147,3,3,1.069167,1.333333,3,1.069167,1.333333,2,2,1.242453
7,3,1117,1,1281,358759,31423,2,21.379999,0,1.098612,3,3,1.069167,1.333333,3,1.069167,1.333333,NaN,NaN,NaN
8,3,1119,1,1457,660918,1250,2,15.280000,0,1.098612,1,1,1.098612,1.000000,1,1.098612,1.000000,NaN,NaN,NaN
9,3,1138,1,1203,402789,972,1,18.980000,0,0.693147,3,3,1.416165,1.000000,3,1.416165,1.000000,NaN,NaN,NaN


E is in columns..


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_proxima,Demanda_uni_equil,lag0tar1,A,B,C,D,E,F,lag1tar1,SA2,L
0,3,1111,1,4402,50789,32846,2,10.920000,0,1.098612,1,1,1.098612,1.000000,1,1.098612,1.000000,NaN,NaN,NaN
1,3,1112,1,2103,1209569,36748,4,46.119999,0,1.609438,1,1,1.609438,1.000000,1,1.609438,1.000000,NaN,NaN,NaN
2,4,1116,1,1603,399077,35147,3,61.500000,0,2.079442,2,2,1.732868,1.000000,2,1.732868,1.000000,1,1,1.386294
3,3,1116,1,2124,174266,36748,3,34.590000,0,1.386294,2,2,1.732868,1.000000,2,1.732868,1.000000,NaN,NaN,NaN
4,4,1117,1,1061,641223,1129,5,88.000000,0,1.791759,3,3,1.069167,1.333333,3,1.069167,1.333333,1,1,1.098612
5,4,1117,1,1272,1649797,1109,1,15.010000,0,0.693147,3,3,1.069167,1.333333,3,1.069167,1.333333,1,1,1.098612
6,5,1117,1,1272,2270968,41938,1,9.910000,0,0.693147,3,3,1.069167,1.333333,3,1.069167,1.333333,2,2,1.242453
7,3,1117,1,1281,358759,31423,2,21.379999,0,1.098612,3,3,1.069167,1.333333,3,1.069167,1.333333,NaN,NaN,NaN
8,3,1119,1,1457,660918,1250,2,15.280000,0,1.098612,1,1,1.098612,1.000000,1,1.098612,1.000000,NaN,NaN,NaN
9,3,1138,1,1203,402789,972,1,18.980000,0,0.693147,3,3,1.416165,1.000000,3,1.416165,1.000000,NaN,NaN,NaN


F is in columns..


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_proxima,Demanda_uni_equil,lag0tar1,A,B,C,D,E,F,lag1tar1,SA2,L
0,3,1111,1,4402,50789,32846,2,10.920000,0,1.098612,1,1,1.098612,1.000000,1,1.098612,1.000000,NaN,NaN,NaN
1,3,1112,1,2103,1209569,36748,4,46.119999,0,1.609438,1,1,1.609438,1.000000,1,1.609438,1.000000,NaN,NaN,NaN
2,4,1116,1,1603,399077,35147,3,61.500000,0,2.079442,2,2,1.732868,1.000000,2,1.732868,1.000000,1,1,1.386294
3,3,1116,1,2124,174266,36748,3,34.590000,0,1.386294,2,2,1.732868,1.000000,2,1.732868,1.000000,NaN,NaN,NaN
4,4,1117,1,1061,641223,1129,5,88.000000,0,1.791759,3,3,1.069167,1.333333,3,1.069167,1.333333,1,1,1.098612
5,4,1117,1,1272,1649797,1109,1,15.010000,0,0.693147,3,3,1.069167,1.333333,3,1.069167,1.333333,1,1,1.098612
6,5,1117,1,1272,2270968,41938,1,9.910000,0,0.693147,3,3,1.069167,1.333333,3,1.069167,1.333333,2,2,1.242453
7,3,1117,1,1281,358759,31423,2,21.379999,0,1.098612,3,3,1.069167,1.333333,3,1.069167,1.333333,NaN,NaN,NaN
8,3,1119,1,1457,660918,1250,2,15.280000,0,1.098612,1,1,1.098612,1.000000,1,1.098612,1.000000,NaN,NaN,NaN
9,3,1138,1,1203,402789,972,1,18.980000,0,0.693147,3,3,1.416165,1.000000,3,1.416165,1.000000,NaN,NaN,NaN


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_proxima,Demanda_uni_equil,lag0tar1,A,B,C,D,E,F,lag1tar1,SA2,L
0,3,1111,1,4402,50789,32846,2,10.920000,0,1.098612,1,1,1.098612,1.000000,1,1.098612,1.000000,NaN,NaN,NaN
1,3,1112,1,2103,1209569,36748,4,46.119999,0,1.609438,1,1,1.609438,1.000000,1,1.609438,1.000000,NaN,NaN,NaN
2,4,1116,1,1603,399077,35147,3,61.500000,0,2.079442,2,2,1.732868,1.000000,2,1.732868,1.000000,1,1,1.386294
3,3,1116,1,2124,174266,36748,3,34.590000,0,1.386294,2,2,1.732868,1.000000,2,1.732868,1.000000,NaN,NaN,NaN
4,4,1117,1,1061,641223,1129,5,88.000000,0,1.791759,3,3,1.069167,1.333333,3,1.069167,1.333333,1,1,1.098612
5,4,1117,1,1272,1649797,1109,1,15.010000,0,0.693147,3,3,1.069167,1.333333,3,1.069167,1.333333,1,1,1.098612
6,5,1117,1,1272,2270968,41938,1,9.910000,0,0.693147,3,3,1.069167,1.333333,3,1.069167,1.333333,2,2,1.242453
7,3,1117,1,1281,358759,31423,2,21.379999,0,1.098612,3,3,1.069167,1.333333,3,1.069167,1.333333,NaN,NaN,NaN
8,3,1119,1,1457,660918,1250,2,15.280000,0,1.098612,1,1,1.098612,1.000000,1,1.098612,1.000000,NaN,NaN,NaN
9,3,1138,1,1203,402789,972,1,18.980000,0,0.693147,3,3,1.416165,1.000000,3,1.416165,1.000000,NaN,NaN,NaN


## Test 3

In [29]:
FE.AddConfigurableFeaturesToTrain(configLag1Target1)

SA2 is in columns..


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_proxima,Demanda_uni_equil,lag0tar1,A,B,C,D,E,F,lag1tar1,SA2,L
0,3,1111,1,4402,50789,32846,2,10.920000,0,1.098612,1,1,1.098612,1.000000,1,1.098612,1.000000,NaN,NaN,NaN
1,3,1112,1,2103,1209569,36748,4,46.119999,0,1.609438,1,1,1.609438,1.000000,1,1.609438,1.000000,NaN,NaN,NaN
2,4,1116,1,1603,399077,35147,3,61.500000,0,2.079442,2,2,1.732868,1.000000,2,1.732868,1.000000,1,1,1.386294
3,3,1116,1,2124,174266,36748,3,34.590000,0,1.386294,2,2,1.732868,1.000000,2,1.732868,1.000000,NaN,NaN,NaN
4,4,1117,1,1061,641223,1129,5,88.000000,0,1.791759,3,3,1.069167,1.333333,3,1.069167,1.333333,1,1,1.098612
5,4,1117,1,1272,1649797,1109,1,15.010000,0,0.693147,3,3,1.069167,1.333333,3,1.069167,1.333333,1,1,1.098612
6,5,1117,1,1272,2270968,41938,1,9.910000,0,0.693147,3,3,1.069167,1.333333,3,1.069167,1.333333,2,2,1.242453
7,3,1117,1,1281,358759,31423,2,21.379999,0,1.098612,3,3,1.069167,1.333333,3,1.069167,1.333333,NaN,NaN,NaN
8,3,1119,1,1457,660918,1250,2,15.280000,0,1.098612,1,1,1.098612,1.000000,1,1.098612,1.000000,NaN,NaN,NaN
9,3,1138,1,1203,402789,972,1,18.980000,0,0.693147,3,3,1.416165,1.000000,3,1.416165,1.000000,NaN,NaN,NaN


Count of missing numbers after SA2 in validation part in column lag1tar1 is 1
L is in columns..


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_proxima,Demanda_uni_equil,lag0tar1,A,B,C,D,E,F,lag1tar1,SA2,L
0,3,1111,1,4402,50789,32846,2,10.920000,0,1.098612,1,1,1.098612,1.000000,1,1.098612,1.000000,NaN,NaN,NaN
1,3,1112,1,2103,1209569,36748,4,46.119999,0,1.609438,1,1,1.609438,1.000000,1,1.609438,1.000000,NaN,NaN,NaN
2,4,1116,1,1603,399077,35147,3,61.500000,0,2.079442,2,2,1.732868,1.000000,2,1.732868,1.000000,1,1,1.386294
3,3,1116,1,2124,174266,36748,3,34.590000,0,1.386294,2,2,1.732868,1.000000,2,1.732868,1.000000,NaN,NaN,NaN
4,4,1117,1,1061,641223,1129,5,88.000000,0,1.791759,3,3,1.069167,1.333333,3,1.069167,1.333333,1,1,1.098612
5,4,1117,1,1272,1649797,1109,1,15.010000,0,0.693147,3,3,1.069167,1.333333,3,1.069167,1.333333,1,1,1.098612
6,5,1117,1,1272,2270968,41938,1,9.910000,0,0.693147,3,3,1.069167,1.333333,3,1.069167,1.333333,2,2,1.242453
7,3,1117,1,1281,358759,31423,2,21.379999,0,1.098612,3,3,1.069167,1.333333,3,1.069167,1.333333,NaN,NaN,NaN
8,3,1119,1,1457,660918,1250,2,15.280000,0,1.098612,1,1,1.098612,1.000000,1,1.098612,1.000000,NaN,NaN,NaN
9,3,1138,1,1203,402789,972,1,18.980000,0,0.693147,3,3,1.416165,1.000000,3,1.416165,1.000000,NaN,NaN,NaN


Count of missing numbers after L in validation part in column lag1tar1 is 1


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_proxima,Demanda_uni_equil,lag0tar1,A,B,C,D,E,F,lag1tar1,SA2,L
0,3,1111,1,4402,50789,32846,2,10.920000,0,1.098612,1,1,1.098612,1.000000,1,1.098612,1.000000,NaN,NaN,NaN
1,3,1112,1,2103,1209569,36748,4,46.119999,0,1.609438,1,1,1.609438,1.000000,1,1.609438,1.000000,NaN,NaN,NaN
2,4,1116,1,1603,399077,35147,3,61.500000,0,2.079442,2,2,1.732868,1.000000,2,1.732868,1.000000,1,1,1.386294
3,3,1116,1,2124,174266,36748,3,34.590000,0,1.386294,2,2,1.732868,1.000000,2,1.732868,1.000000,NaN,NaN,NaN
4,4,1117,1,1061,641223,1129,5,88.000000,0,1.791759,3,3,1.069167,1.333333,3,1.069167,1.333333,1,1,1.098612
5,4,1117,1,1272,1649797,1109,1,15.010000,0,0.693147,3,3,1.069167,1.333333,3,1.069167,1.333333,1,1,1.098612
6,5,1117,1,1272,2270968,41938,1,9.910000,0,0.693147,3,3,1.069167,1.333333,3,1.069167,1.333333,2,2,1.242453
7,3,1117,1,1281,358759,31423,2,21.379999,0,1.098612,3,3,1.069167,1.333333,3,1.069167,1.333333,NaN,NaN,NaN
8,3,1119,1,1457,660918,1250,2,15.280000,0,1.098612,1,1,1.098612,1.000000,1,1.098612,1.000000,NaN,NaN,NaN
9,3,1138,1,1203,402789,972,1,18.980000,0,0.693147,3,3,1.416165,1.000000,3,1.416165,1.000000,NaN,NaN,NaN


## Test 4

In [40]:
FE.AddConfigurableFeaturesToTrain(configLag1Target1F)

SAP1CC is not in columns..


SAP1CC
Semana Agencia_ID Producto_ID        
4      1111       32846             1
       1112       36748             1
       1116       36748             1
       1117       31423             1
       1119       1250              1
       1123       37058             1
       1127       3270              1
                  31423             1
       1138       972               1
5      1116       35147             1
       1117       1109              1
                  1129              1
       1138       972               1
6      1117       41938             1
       1123       4767              1
       1138       972               1

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_proxima,Demanda_uni_equil,...,B,C,D,E,F,lag1tar1,SA2,L,Lag1Target1F,SAP1CC
0,3,1111,1,4402,50789,32846,2,10.920000,0,1.098612,...,1.098612,1.000000,1,1.098612,1.000000,NaN,NaN,NaN,NaN,NaN
1,3,1112,1,2103,1209569,36748,4,46.119999,0,1.609438,...,1.609438,1.000000,1,1.609438,1.000000,NaN,NaN,NaN,NaN,NaN
2,4,1116,1,1603,399077,35147,3,61.500000,0,2.079442,...,1.732868,1.000000,2,1.732868,1.000000,1,1,1.386294,NaN,NaN
3,3,1116,1,2124,174266,36748,3,34.590000,0,1.386294,...,1.732868,1.000000,2,1.732868,1.000000,NaN,NaN,NaN,NaN,NaN
4,4,1117,1,1061,641223,1129,5,88.000000,0,1.791759,...,1.069167,1.333333,3,1.069167,1.333333,1,1,1.098612,NaN,NaN
5,4,1117,1,1272,1649797,1109,1,15.010000,0,0.693147,...,1.069167,1.333333,3,1.069167,1.333333,1,1,1.098612,NaN,NaN
6,5,1117,1,1272,2270968,41938,1,9.910000,0,0.693147,...,1.069167,1.333333,3,1.069167,1.333333,2,2,1.242453,NaN,NaN
7,3,1117,1,1281,358759,31423,2,21.379999,0,1.098612,...,1.069167,1.333333,3,1.069167,1.333333,NaN,NaN,NaN,NaN,NaN
8,3,1119,1,1457,660918,1250,2,15.280000,0,1.098612,...,1.098612,1.000000,1,1.098612,1.000000,NaN,NaN,NaN,NaN,NaN
9,3,1138,1,1203,402789,972,1,18.980000,0,0.693147,...,1.416165,1.000000,3,1.416165,1.000000,NaN,NaN,NaN,NaN,NaN


Count of missing numbers after SAP1CC in validation part in column Lag1Target1F is 1
SAP1MM is not in columns..


SAP1MM
Semana Agencia_ID Producto_ID          
4      1111       32846        1.098612
       1112       36748        1.609438
       1116       36748        1.386294
       1117       31423        1.098612
       1119       1250         1.098612
       1123       37058        2.302585
       1127       3270         2.564949
                  31423        1.098612
       1138       972          0.693147
5      1116       35147        2.079442
       1117       1109         0.693147
                  1129         1.791759
       1138       972          1.609438
6      1117       41938        0.693147
       1123       4767         0.693147
       1138       972          1.945910

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_proxima,Demanda_uni_equil,...,B,C,D,E,F,lag1tar1,SA2,L,Lag1Target1F,SAP1MM
0,3,1111,1,4402,50789,32846,2,10.920000,0,1.098612,...,1.098612,1.000000,1,1.098612,1.000000,NaN,NaN,NaN,NaN,NaN
1,3,1112,1,2103,1209569,36748,4,46.119999,0,1.609438,...,1.609438,1.000000,1,1.609438,1.000000,NaN,NaN,NaN,NaN,NaN
2,4,1116,1,1603,399077,35147,3,61.500000,0,2.079442,...,1.732868,1.000000,2,1.732868,1.000000,1,1,1.386294,NaN,NaN
3,3,1116,1,2124,174266,36748,3,34.590000,0,1.386294,...,1.732868,1.000000,2,1.732868,1.000000,NaN,NaN,NaN,NaN,NaN
4,4,1117,1,1061,641223,1129,5,88.000000,0,1.791759,...,1.069167,1.333333,3,1.069167,1.333333,1,1,1.098612,NaN,NaN
5,4,1117,1,1272,1649797,1109,1,15.010000,0,0.693147,...,1.069167,1.333333,3,1.069167,1.333333,1,1,1.098612,NaN,NaN
6,5,1117,1,1272,2270968,41938,1,9.910000,0,0.693147,...,1.069167,1.333333,3,1.069167,1.333333,2,2,1.242453,NaN,NaN
7,3,1117,1,1281,358759,31423,2,21.379999,0,1.098612,...,1.069167,1.333333,3,1.069167,1.333333,NaN,NaN,NaN,NaN,NaN
8,3,1119,1,1457,660918,1250,2,15.280000,0,1.098612,...,1.098612,1.000000,1,1.098612,1.000000,NaN,NaN,NaN,NaN,NaN
9,3,1138,1,1203,402789,972,1,18.980000,0,0.693147,...,1.416165,1.000000,3,1.416165,1.000000,NaN,NaN,NaN,NaN,NaN


Count of missing numbers after SAP1MM in validation part in column Lag1Target1F is 1


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_proxima,Demanda_uni_equil,...,A,B,C,D,E,F,lag1tar1,SA2,L,Lag1Target1F
0,3,1111,1,4402,50789,32846,2,10.920000,0,1.098612,...,1,1.098612,1.000000,1,1.098612,1.000000,NaN,NaN,NaN,NaN
1,3,1112,1,2103,1209569,36748,4,46.119999,0,1.609438,...,1,1.609438,1.000000,1,1.609438,1.000000,NaN,NaN,NaN,NaN
2,4,1116,1,1603,399077,35147,3,61.500000,0,2.079442,...,2,1.732868,1.000000,2,1.732868,1.000000,1,1,1.386294,NaN
3,3,1116,1,2124,174266,36748,3,34.590000,0,1.386294,...,2,1.732868,1.000000,2,1.732868,1.000000,NaN,NaN,NaN,NaN
4,4,1117,1,1061,641223,1129,5,88.000000,0,1.791759,...,3,1.069167,1.333333,3,1.069167,1.333333,1,1,1.098612,NaN
5,4,1117,1,1272,1649797,1109,1,15.010000,0,0.693147,...,3,1.069167,1.333333,3,1.069167,1.333333,1,1,1.098612,NaN
6,5,1117,1,1272,2270968,41938,1,9.910000,0,0.693147,...,3,1.069167,1.333333,3,1.069167,1.333333,2,2,1.242453,NaN
7,3,1117,1,1281,358759,31423,2,21.379999,0,1.098612,...,3,1.069167,1.333333,3,1.069167,1.333333,NaN,NaN,NaN,NaN
8,3,1119,1,1457,660918,1250,2,15.280000,0,1.098612,...,1,1.098612,1.000000,1,1.098612,1.000000,NaN,NaN,NaN,NaN
9,3,1138,1,1203,402789,972,1,18.980000,0,0.693147,...,3,1.416165,1.000000,3,1.416165,1.000000,NaN,NaN,NaN,NaN


## Setup prod

In [10]:
parameterDict =       {"ValidationStart":6, 
 "ValidationSmallerThan":7,
   "maxLag":2,
    "trainHdfPath":'../../input/train.h5',
    "trainHdfFile":"train",
    "testHdfPath":"../../input/test.h5",
    "testHdfFile":"test", 
    "trainTypes" : {'Semana':np.uint8, 'Agencia_ID':np.uint16, 'Canal_ID':np.uint8,'Ruta_SAK':np.uint16, 
        'Cliente_ID':np.uint32, 'Producto_ID':np.uint16,'Venta_uni_hoy':np.uint16, 'Venta_hoy':np.float32,
                    'Dev_uni_proxima': np.uint32, 'Dev_proxima':np.float32,'Demanda_uni_equil':np.uint32}, 
    "testTypes" : {'id':np.uint32,'Semana':np.uint8, 'Agencia_ID':np.uint16, 'Canal_ID':np.uint8,'Ruta_SAK':np.uint16,
        'Cliente_ID':np.uint32, 'Producto_ID':np.uint16},
    "trainCsvPath":'../../input/train.csv'   ,
    "testCsvPath":'../../input/test.csv'}

FE = FeatureEngineering(**parameterDict)
print FE.__dict__

{'maxLag': 2, 'testTypes': {'Cliente_ID': <type 'numpy.uint32'>, 'Ruta_SAK': <type 'numpy.uint16'>, 'Canal_ID': <type 'numpy.uint8'>, 'Producto_ID': <type 'numpy.uint16'>, 'Agencia_ID': <type 'numpy.uint16'>, 'Semana': <type 'numpy.uint8'>, 'id': <type 'numpy.uint32'>}, 'testHdfFile': 'test', 'trainTypes': {'Dev_proxima': <type 'numpy.float32'>, 'Venta_uni_hoy': <type 'numpy.uint16'>, 'Cliente_ID': <type 'numpy.uint32'>, 'Demanda_uni_equil': <type 'numpy.uint32'>, 'Ruta_SAK': <type 'numpy.uint16'>, 'Canal_ID': <type 'numpy.uint8'>, 'Venta_hoy': <type 'numpy.float32'>, 'Producto_ID': <type 'numpy.uint16'>, 'Agencia_ID': <type 'numpy.uint16'>, 'Dev_uni_proxima': <type 'numpy.uint32'>, 'Semana': <type 'numpy.uint8'>}, 'trainCsvPath': '../../input/train.csv', 'testHdfPath': '../../input/test.h5', 'testCsvPath': '../../input/test.csv', 'ValidationStart': 6, 'trainHdfFile': 'train', 'trainHdfPath': '../../input/train.h5', 'ValidationSmallerThan': 7}


## Train and test data, from csv to hdf while log transformation on Demanda..

In [11]:
#FE.ReadCsv('both')
#FE.Preprocess('train', [["Demanda_uni_equil",np.log1p,'float32']])
#FE.SaveDataFrameToHdf('both')

## Read test and train data from hdf..

In [4]:
FE.ReadHdf('both')

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Demanda_uni_equil
0,3,1110,7,3301,15766,1212,1.386294
1,3,1110,7,3301,15766,1216,1.609438


Semana                 uint8
Agencia_ID            uint16
Canal_ID               uint8
Ruta_SAK              uint16
Cliente_ID            uint32
Producto_ID           uint16
Demanda_uni_equil    float32
dtype: object
<class 'pandas.core.frame.DataFrame'>
Int64Index: 74180464 entries, 0 to 74180463
Data columns (total 7 columns):
Semana               uint8
Agencia_ID           uint16
Canal_ID             uint8
Ruta_SAK             uint16
Cliente_ID           uint32
Producto_ID          uint16
Demanda_uni_equil    float32
dtypes: float32(1), uint16(3), uint32(1), uint8(2)
memory usage: 1.4 GB
None


,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID
0,0,11,4037,1,2209,4639078,35305
1,1,11,2237,1,1226,4705135,1238


id             uint32
Semana          uint8
Agencia_ID     uint16
Canal_ID        uint8
Ruta_SAK       uint16
Cliente_ID     uint32
Producto_ID    uint16
dtype: object
<class 'pandas.core.frame.DataFrame'>
Int64Index: 6999251 entries, 0 to 6999250
Data columns (total 7 columns):
id             uint32
Semana         uint8
Agencia_ID     uint16
Canal_ID       uint8
Ruta_SAK       uint16
Cliente_ID     uint32
Producto_ID    uint16
dtypes: uint16(3), uint32(2), uint8(2)
memory usage: 133.5 MB
None


## Test setup..

In [22]:
parameterDict = {"ValidationStart":6, 
    "ValidationSmallerThan":7,
    "maxLag":2,    
    "trainHdfPath":'../../input/train_100.h5',
    "trainHdfFile":"train",
    "testHdfPath":"../../input/test.h5",
    "testHdfFile":"test", 
    "trainTypes" : {'Semana':np.uint8, 'Agencia_ID':np.uint16, 'Canal_ID':np.uint8,'Ruta_SAK':np.uint16, 
        'Cliente_ID':np.uint32, 'Producto_ID':np.uint16,'Venta_uni_hoy':np.uint16, 'Venta_hoy':np.float32,
                    'Dev_proxima':np.float32,'Demanda_uni_equil':np.uint32}, 
    "testTypes" : {'id':np.uint32,'Semana':np.uint8, 'Agencia_ID':np.uint16, 'Canal_ID':np.uint8,'Ruta_SAK':np.uint16,
        'Cliente_ID':np.uint32, 'Producto_ID':np.uint16},
    "trainCsvPath":'../../input/train_100.csv'   ,
    "testCsvPath":'../../input/test.csv'}

FE = FeatureEngineering(**parameterDict)
print FE.__dict__

{'maxLag': 2, 'testTypes': {'Cliente_ID': <type 'numpy.uint32'>, 'Ruta_SAK': <type 'numpy.uint16'>, 'Canal_ID': <type 'numpy.uint8'>, 'Producto_ID': <type 'numpy.uint16'>, 'Agencia_ID': <type 'numpy.uint16'>, 'Semana': <type 'numpy.uint8'>, 'id': <type 'numpy.uint32'>}, 'testHdfFile': 'test', 'trainTypes': {'Cliente_ID': <type 'numpy.uint32'>, 'Demanda_uni_equil': <type 'numpy.uint32'>, 'Producto_ID': <type 'numpy.uint16'>, 'Agencia_ID': <type 'numpy.uint16'>, 'Dev_proxima': <type 'numpy.float32'>, 'Semana': <type 'numpy.uint8'>, 'Ruta_SAK': <type 'numpy.uint16'>, 'Canal_ID': <type 'numpy.uint8'>, 'Venta_uni_hoy': <type 'numpy.uint16'>, 'Venta_hoy': <type 'numpy.float32'>}, 'trainCsvPath': '../../input/train_100.csv', 'testHdfPath': '../../input/test.h5', 'testCsvPath': '../../input/test.csv', 'ValidationStart': 6, 'trainHdfFile': 'train', 'trainHdfPath': '../../input/train_100.h5', 'ValidationSmallerThan': 7}


In [23]:
#FE.ReadCsv('train')
#FE.Preprocess('train', [["Demanda_uni_equil",np.log1p,'float32']])
#FE.SaveDataFrameToHdf('train')

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_proxima,Demanda_uni_equil
0,3,1111,1,4402,50789,32846,2,10.920000,0,2
1,3,1112,1,2103,1209569,36748,4,46.119999,0,4


<class 'pandas.core.frame.DataFrame'>
Int64Index: 18 entries, 0 to 17
Data columns (total 10 columns):
Semana               18 non-null uint8
Agencia_ID           18 non-null uint16
Canal_ID             18 non-null uint8
Ruta_SAK             18 non-null uint16
Cliente_ID           18 non-null uint32
Producto_ID          18 non-null uint16
Venta_uni_hoy        18 non-null uint16
Venta_hoy            18 non-null float32
Dev_proxima          18 non-null float32
Demanda_uni_equil    18 non-null uint32
dtypes: float32(2), uint16(4), uint32(2), uint8(2)
memory usage: 612.0 bytes
None


In [24]:
FE.ReadHdf('train')

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_proxima,Demanda_uni_equil
0,3,1111,1,4402,50789,32846,2,10.920000,0,1.098612
1,3,1112,1,2103,1209569,36748,4,46.119999,0,1.609438


<class 'pandas.core.frame.DataFrame'>
Int64Index: 18 entries, 0 to 17
Data columns (total 10 columns):
Semana               18 non-null uint8
Agencia_ID           18 non-null uint16
Canal_ID             18 non-null uint8
Ruta_SAK             18 non-null uint16
Cliente_ID           18 non-null uint32
Producto_ID          18 non-null uint16
Venta_uni_hoy        18 non-null uint16
Venta_hoy            18 non-null float32
Dev_proxima          18 non-null float32
Demanda_uni_equil    18 non-null float32
dtypes: float32(3), uint16(4), uint32(1), uint8(2)
memory usage: 540.0 bytes
None


In [12]:
display(FE.AddDemandaGeneralMean())

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Demanda_uni_equil,DemandaGeneralMean
0,3,1,7,3301,15766,1200,1.386294,5.159912
1,3,2,7,3301,15766,1200,1.609438,5.159912
2,3,1,7,3301,15766,1200,3.713572,5.159912
3,3,2,7,3301,15766,1300,1.098612,5.159912
4,3,3,7,3301,15900,1300,1.386294,5.159912
5,3,2,7,3301,15766,1300,6.908755,5.159912
6,4,3,7,3301,15766,1200,11.512936,5.159912
7,4,2,7,3301,15766,1300,11.512936,5.159912
8,4,1,7,3301,15766,1300,11.512936,5.159912
9,4,4,7,3301,15900,1400,11.512936,5.159912


None

In [28]:
qw = FE.train.loc[FE.train.loc[:,'Semana']>4]
qw


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Demanda_uni_equil,DemandaGeneralMean,lag2A,lag2Ca,lag2R,lag2Cl,lag2P,lag2S
10,5,1110,7,3301,15766,1200,2.397895,5.159912,2.683828,2.683828,2.683828,2.943334,2.236435,2.683828
11,5,1110,7,3301,15766,1200,2.397895,5.159912,2.683828,2.683828,2.683828,2.943334,2.236435,2.683828
12,5,1110,7,3301,15766,1200,2.890372,5.159912,2.683828,2.683828,2.683828,2.943334,2.236435,2.683828
13,5,1110,7,3301,15900,1400,2.397895,5.159912,2.683828,2.683828,2.683828,1.386294,NaN,2.683828
14,6,1110,7,3301,15766,1400,5.298317,5.159912,11.512936,11.512936,11.512936,11.512936,11.512936,11.512936


In [79]:


FE.train.loc[:,'target'] = np.nan
FE.train

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Demanda_uni_equil,DemandaGeneralMean,lag2A,lag2Ca,lag2R,lag2Cl,lag2P,lag2S,target
0,3,1110,7,3301,15766,1200,1.386294,5.159912,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,1110,7,3301,15766,1200,1.609438,5.159912,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,1110,7,3301,15766,1200,3.713572,5.159912,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,1110,7,3301,15766,1300,1.098612,5.159912,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3,1110,7,3301,15900,1300,1.386294,5.159912,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,3,1110,7,3301,15766,1300,6.908755,5.159912,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,4,1110,7,3301,15766,1200,11.512936,5.159912,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,4,1110,7,3301,15766,1300,11.512936,5.159912,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,4,1110,7,3301,15766,1300,11.512936,5.159912,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,4,1110,7,3301,15900,1400,11.512936,5.159912,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [82]:
FE.train.loc[pd.isnull(FE.train['target']), 'target'] = FE.train.loc[pd.isnull(FE.train['target']), 'lag2P']
FE.train["target"]

0           NaN
1           NaN
2           NaN
3           NaN
4           NaN
5           NaN
6           NaN
7           NaN
8           NaN
9           NaN
10     2.236435
11     2.236435
12     2.236435
13          NaN
14    11.512936
Name: target, dtype: float32

In [87]:
FE.train.loc[FE.train['Semana']>4,"target"].isnull().sum()

10     2.236435
11     2.236435
12     2.236435
13          NaN
14    11.512936
Name: target, dtype: float32

In [88]:
FE.train.loc[FE.train['Semana']>4,"target"].isnull().sum()

1

In [89]:
FE.train.loc[pd.isnull(FE.train['target']), 'target'] = FE.train.loc[pd.isnull(FE.train['target']), 'lag2Cl']
FE.train["target"]

0           NaN
1           NaN
2           NaN
3           NaN
4           NaN
5           NaN
6           NaN
7           NaN
8           NaN
9           NaN
10     2.236435
11     2.236435
12     2.236435
13     1.386294
14    11.512936
Name: target, dtype: float32

In [90]:
FE.train.loc[FE.train['Semana']>4,"target"].isnull().sum()

0

In [45]:
"Semana" not in FE.train.columns

False

In [46]:
FE.train

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Demanda_uni_equil,lag0par1,lag2par1
0,3,1,7,3301,15766,1200,1.386294,5.537601,NaN
1,3,2,7,3301,15766,1200,1.609438,4.320922,NaN
2,3,1,7,3301,15766,1200,3.713572,5.537601,NaN
3,3,2,7,3301,15766,1300,1.098612,4.320922,NaN
4,3,3,7,3301,15900,1300,1.386294,4.546874,NaN
5,3,2,7,3301,15766,1300,6.908755,4.320922,NaN
6,4,3,7,3301,15766,1200,11.512936,4.546874,NaN
7,4,2,7,3301,15766,1300,11.512936,4.320922,NaN
8,4,1,7,3301,15766,1300,11.512936,5.537601,NaN
9,4,4,7,3301,15900,1400,11.512936,11.512936,NaN


In [64]:
print "{} haha {}".format(1,2)

1 haha 2


In [25]:
for i in FE.train.groupby('Canal_ID')['Demanda_uni_equil']:
    print i


(2, 13    2.397895
14    5.298317
Name: Demanda_uni_equil, dtype: float32)
(7, 0      1.386294
1      1.609438
2      3.713572
3      1.098612
4      1.386294
5      6.908755
6     11.512936
7     11.512936
8     11.512936
9     11.512936
10     2.397895
11     2.397895
12     2.890372
15     2.995732
17     4.595120
18     5.298317
Name: Demanda_uni_equil, dtype: float32)
(10, 16    5.288267
Name: Demanda_uni_equil, dtype: float32)


In [29]:
FE.train[['Canal_ID','Demanda_uni_equil']]



,Canal_ID,Demanda_uni_equil
0,7,1.386294
1,7,1.609438
2,7,3.713572
3,7,1.098612
4,7,1.386294
5,7,6.908755
6,7,11.512936
7,7,11.512936
8,7,11.512936
9,7,11.512936


In [31]:
["Cliente_ID","Producto_ID"] + 'abc'

TypeError: can only concatenate list (not "str") to list

In [ ]:
import xgboost as xgb

params = {}
params['objective'] = "reg:linear"
params['eta'] = 0.1
params['max_depth'] = 5
params['subsample'] = 0.8
params['colsample_bytree'] = 0.6
params['silent'] = True
'alpha': 0.0001, 'lambda': 1

In [27]:
import xgboost as xgb

In [28]:
regres = xgb.XGBRegressor()

In [29]:
regres

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)